Created by Marina Dunn, Spring 2022

**Resources used:**

https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2

https://github.com/deepskies/deepmerge-public/blob/master/DeepMerge-noisy.ipynb

https://towardsdatascience.com/convolutional-neural-networks-explained-9cc5188c4939

https://towardsdatascience.com/cross-entropy-loss-function-f38c4ec8643e

https://www.superdatascience.com/blogs/convolutional-neural-networks-cnn-softmax-crossentropy

In [ ]:
# system level
import os
import numpy as np
from numpy import random
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
from __future__ import print_function
import time
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# machine learning
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import VGG16
#from keras.utils import np_utils
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical

from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, brier_score_loss
from sklearn.model_selection import cross_val_score, KFold

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import matplotlib.patches as mpatches
# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'
import matplotlib
%matplotlib inline

In [2]:
tf.config.run_functions_eagerly(True)

### Load and define data

In [3]:
# Load year 1 train, test, validation image files
X_train_1 = np.load('images_Y1_train.npy')
X_test_1 = np.load('images_Y1_test.npy')
X_valid_1 = np.load('images_Y1_valid.npy')

In [4]:
# Load year 10 train, test, validation image files
X_train_10 = np.load('images_Y10_train.npy')
X_test_10 = np.load('images_Y10_test.npy')
X_valid_10 = np.load('images_Y10_valid.npy')

In [5]:
# Load year 1 and 10, label small subset test files
X_test_1_sub = np.load('images_Y1_test_150.npy')
X_test_10_sub = np.load('images_Y10_test_150.npy')
Y_test_sub = np.load('labels_test_150.npy')

In [6]:
# Load image label files
Y_train_label = np.load('labels_train.npy')
Y_test_label = np.load('labels_test.npy')
Y_valid_label = np.load('labels_valid.npy')

In [7]:
# define full datasets by combining train + test + validation sets
X_1_data = np.concatenate([X_train_1, X_test_1, X_valid_1], axis=0)
np.shape(X_1_data)

(33557, 3, 100, 100)

In [8]:
X_10_data = np.concatenate([X_train_10, X_test_10, X_valid_10], axis=0)
np.shape(X_10_data)

(33557, 3, 100, 100)

In [9]:
Y_data = np.concatenate([Y_train_label, Y_test_label, Y_valid_label], axis=0)
np.shape(Y_data)

(33557, 3)

**Choosing batch size, epoch:**

batch size: larger batch sizes result in faster progress in training, but don't always converge as fast. Smaller batch sizes train slower, but can converge faster. 
epoch: models improve with more epochs of training, to a point; will start to plateau in accuracy as they converge. 

In [10]:
# training variables
# 70:10:20 for training:valid:test
NUM_TRAIN = 23487 # num train images
NUM_TEST = 6715 # num test images
NUM_VALIDATION = 3355 # num validation images
shuffle = True
NUM_EPOCH = 500
batch_size = 128

In [11]:
# Shuffle data
random.seed(5)
idx = np.random.choice(len(X_1_data), size=len(X_1_data), replace=False)
X_1_data = X_1_data[idx]
X_10_data = X_10_data[idx]
Y_data = Y_data[idx]

In [12]:
# slice the image arrays

ind_valid_start = ind_train_end = NUM_TRAIN
ind_valid_end = ind_test_start = NUM_TRAIN + NUM_VALIDATION
ind_test_end = NUM_TRAIN + NUM_VALIDATION + NUM_TEST

X_train_1 = X_1_data[:ind_train_end, :, :, :]
X_valid_1 = X_1_data[ind_valid_start: ind_valid_end, :, :, :]
X_test_1 = X_1_data[ind_test_start: ind_test_end, :, :, :]

X_train_10 = X_10_data[:ind_train_end, :, :, :]
X_valid_10 = X_10_data[ind_valid_start: ind_valid_end, :, :, :]
X_test_10 = X_10_data[ind_test_start: ind_test_end, :, :, :]

# slice the label arrays
Y_train = Y_data[:ind_train_end]
Y_valid = Y_data[ind_valid_start: ind_valid_end]
Y_test = Y_data[ind_test_start: ind_test_end]

Y_data = Y_data[:, 0]
Y_train = Y_train[:, 0]
Y_valid = Y_valid[:, 0]
Y_test = Y_test[:, 0]

In [13]:
# check data sizes
NUM_TOTAL = NUM_TRAIN + NUM_TEST + NUM_VALIDATION
print(NUM_TOTAL)
assert NUM_TOTAL == len(X_train_1) + len(X_test_1) + len(X_valid_1), "total training, test, validation samples not equal to total samples - exiting"


33557


### Exploratory Data Analysis

In [14]:
# Y_train_df = pd.DataFrame(Y_train_label)
# Y_test_df = pd.DataFrame(Y_test_label)
# Y_valid_df = pd.DataFrame(Y_valid_label)

In [15]:
#Y_train_df.info()
#Y_test_df.info()
#Y_valid_df.info()

In [16]:
#Y_train_df.value_counts()
#Y_test_df.value_counts()
#Y_valid_df.value_counts()

In [17]:
#Y_train_df.describe()
#Y_test_df.describe()
#Y_valid_df.describe()

Each image represents a series of pixels in a grid-like format with a value of brightness for each pixel.

In [18]:
print( "Data dimensions: ")
# training on year 1
print( "Input data: ", np.shape(X_1_data), np.shape(Y_data)) # same for year 10
print( "Training Set: ", np.shape(X_train_1), np.shape(Y_train)) # same for year 10
print( "Test Set: ", np.shape(X_test_1), np.shape(Y_test)) # same for year 10
print( "Validation Set: ", np.shape(X_valid_1), np.shape(Y_valid)) # same for year 10

Data dimensions: 
Input data:  (33557, 3, 100, 100) (33557,)
Training Set:  (23487, 3, 100, 100) (23487,)
Test Set:  (6715, 3, 100, 100) (6715,)
Validation Set:  (3355, 3, 100, 100) (3355,)


In [19]:
print("Subset Test Set: ", np.shape(X_test_1_sub))

Subset Test Set:  (150, 3, 100, 100)


```
Multi-Class Classification
3 columns: spiral (or '0'), elliptical ('1'), or a galaxy merger ('2’) - (num_classes – 1)
No null values, min:0, max:1
dtype: int64

(year 1)
Training set details:
• 23487 total images
• 10017 Spiral galaxies (mean: 0.426491, std: 0.494577)
• 5705 Elliptical galaxies (mean: 0.242900, std: 0.428844)
• 7765 Merger galaxies (mean: 0.330608, std: 0.470442)
• Memory usage: 550.6 KB

Test set details:
• 6715 total images
• 2863 Spiral galaxies (mean: 0.426359, std: 0.494584)
• 1631 Elliptical galaxies (mean: 0.242889, std: 0.428861)
• 2221 Merger galaxies (mean: 0.330752, std: 0.470519)
• Memory usage: 157.5 KB

Validation set details:
• 3355 total images
• 1432 Spiral galaxies (mean: 0.426826, std: 0.494690)
• 815 Elliptical galaxies (mean: 0.242921, std: 0.428912)
• 1108 Merger galaxies (mean: 0.330253, std: 0.470374)
• Memory usage: 78.8 KB
```

In [20]:
# cast data types as floats
X_train_1 = X_train_1.astype('float32')
X_test_1 = X_test_1.astype('float32')
X_valid_1 = X_valid_1.astype('float32')

X_train_10 = X_train_10.astype('float32')
X_test_10 = X_test_10.astype('float32')
X_valid_10 = X_valid_10.astype('float32')

X_test_1_sub = X_test_1_sub.astype('float32')
X_test_10_sub = X_test_10_sub.astype('float32')

In [ ]:
# Visualize images

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

### Build CNN model:
First build a standard deterministic CNN classifier model as a baseline model. CNN/ConvNet typically uses ~3 convolution layers: a convolutional layer, a pooling layer, and a fully connected layer. The layer setup is such that simpler patterns are recognized first, then increasingly complex patterns thereafter.

Perform series of convolution + pooling operations, then a number of fully connected layers. 

**Convolutional Layer:** Contains most of the network's main computational load. Performs dot product between kernel (matrix 1) and restricted portion of the receptive field (matrix 2). Forward pass. Creates an activation/feature map, or 2D representation of the image.

**Pooling Layer:** provides summary statistic of nearby outputs at various network output locations. Pooling operation performed individually on every slice of activation map. Max pooling is most popular.

**Fully Connected Layer:** helps map the representation b/w input and the output. Matrix multiplication, followed by a bias effect. Helps to learn non-linear combinations of features.

#### Steps:

1) Train

2) Validate on a holdout set generated from the original training data

3) Evaluate on the test data

In [21]:
print("Tensorflow Version: ", tf.__version__)
print("Tensorflow Probability Version: ", tfp.__version__)

Tensorflow Version:  2.8.0
Tensorflow Probability Version:  0.16.0


#### Defining CNN model architecture:

##### Activation functions:

**-Sigmoid:** σ(κ) = 1/(1+e¯κ), takes real-value number & “squashes” into a range 0-1

**-ReLU/'relu':** Rectified Linear Unit, calculates ƒ(κ)=max (0,κ), or the activation is simply threshold at 0. More reliable + faster than sigmoid, tanh. More non-linearity gives more power to the model

**-Softmax/Normalized Exponential:** converts a vector of numbers into a vector of probabilities, where the probabilities of each value are proportional to the relative scale of each value in the vector.

##### Keras layers:

**Convolution2D():** 2D convolution layer (e.g. spatial convolution over images). 

-args: activation (activation function to use), 

**MaxPooling2D():** Max pooling operation for 2D spatial data

-args: pool_size (window size over which to take the maximum), strides (Specifies how far the pooling window moves for each pooling step; sliding size of the kernel), padding (''valid"- no padding, or ''same"- padding evenly to the left/right or up/down of the input such that output has the same height/width dimension as the input), data_format (ordering of the dimensions in the inputs)

data_format='channels_first': 

input: 4D tensor with shape (batch_size, channels, rows, cols)
output: 4D tensor with shape (batch_size, channels, pooled_rows, pooled_cols).

**Dense():** regular densely-connected neural network layer (describes a layer of neurons)

-args: activation (activation function to use), kernel_regularizer (regularizer function applied to the kernel weights matrix)

##### Regularization
**Dropout:** Most popular regularization technique, prevents overfitting. At each iteration, neuron temporarily “dropped” with probability 'p'. This neuron is resampled with 'p' at every training step, so will be active at next step. Typically ~0.5 or 50%. Can be applied to input or hidden layer nodes, but not output nodes. 

**Batch Normalization:** explicitly forces the activations throughout a network to take on a Gaussian distribution at the beginning of training, applying a transformation that maintains mean output close to 0 and output standard deviation close to 1. BatchNorm layer inserted immediately after fully connected layers, or convolutional layers, and before non-linearities. Significantly more robust to bad initialization.

##### Keras Early stopping
**Early stopping:** Early stopping call back function can be used to monitor either accuracy or loss, stopping when there's either a loss increment, or accuracy decrement. Should monitor the thing that is more sensitive, which is usually loss function- will start seeing signs of overfitting in loss earlier. 

##### Keras Optimizers:  
**Adam:** implements the Adam algorithm; learning rate of 0.001. Stochastic gradient descent method based on adaptive estimation of 1st-order and 2nd-order moments. Very performant.

##### Loss function: 
**Categorical Cross-entropy:** Computes the cross-entropy loss between the labels and predictions. Used when adjusting model weights during training. Categorical cross-entropy is used with 'softmax' activation for multi-class classification problem, i.e. when true label values are one-hot encoded for 3-class classification problem.

**Sparse Categorical Cross-entropy:** Same as above, but does not require one-hot encoding. Used when there are many labels.

##### Additional techniques to potentially look at:
**Cross validation:** (i.e. k-fold), partition the original training data set into k equal subsets, each called a fold (named f1, f2, …, fk, for i = 1 to i = k). Keep the fold fi as the Validation set, and keep all the remaining k-1 folds in the Cross validation training set. Train model using the cross validation training set; calculate accuracy by validating predicted results against the validation set. Estimate accuracy of model by averaging the accuracies derived in all the k cases of cross validation. Helps reduce overfitting problem.

In [22]:
class_names = ['Spiral', 'Elliptical', 'Merger']
NUM_CLASSES = len(class_names)
print(class_names)

['Spiral', 'Elliptical', 'Merger']


Image -> Conv layer -> Pool Layer -> Conv layer -> Pool Layer - > Conv layer -> Pool Layer - > Flatten -> Dense -> Dense-> Dense-> Dense-> Output

In [25]:
# Defining model layers, build with the Functional API
data_shape = np.shape(X_1_data)
#(channels, rows, cols)
input_shape = (3, 100, 100)

# Constraints for layer 1
# input: tensors of shape (color_channels, image_height, image_width), ignoring batch size
x = Input(shape=input_shape)
#For CNN: Convolution typically uses 3x3 windows, stride 1 & with padding. Will try 5x5 window first to attempt tp catch more features, but try multiple
c0 = Conv2D(8, (5, 5), activation='relu', strides=(1, 1), padding='same', data_format='channels_first', name='conv_1')(x)
b0 = BatchNormalization()(c0)
#For CNN: pooling typically uses 2x2 windows, stride 2 & no padding
d0 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first', name='maxpool_1')(b0)
e0 = Dropout(0.5)(d0)

# Constraints for layer 2
c1 = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), padding='same', data_format='channels_first', name='conv_2')(e0)
b1 = BatchNormalization()(c1)
d1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first', name='maxpool_2')(b1)
e1 = Dropout(0.5)(d1)

# Constraints for layer 3
c2 = Conv2D(32, (3, 3), activation='relu', strides=(1, 1), padding='same', data_format='channels_first', name='conv_3')(e1)
b2 = BatchNormalization()(c2)
d2 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format='channels_first', name='maxpool_3')(b2)
e2 = Dropout(0.5)(d2)


# Constraints for layer 4
#flatten (or unroll) the 3D output to 1D
f = Flatten()(e2)
# Dense layer: takes 1D vector as input, current output is a 3D tensor; start simple, then change unit to try to increase performance
z0 = Dense(512, activation='relu', name='dense_1')(f)
z1 = Dense(256, activation='relu', name='dense_2')(z0)
# use L2 regulization: take sum of all params squared & add it w/ square difference of actual output & predictions
# weights not sparse, will get much better accuracy than L1
z2 = Dense(64, activation='relu', kernel_regularizer=l2(0.0001), name='dense_3')(z1)
z3 = Dense(32, activation='relu', kernel_regularizer=l2(0.0001), name='dense_4')(z2)
# output layer must create 3 output values, one for each class
# use “softmax” activation function to ensure output values are in range 0-1 & may be used as predicted probabilities
y = Dense(3, activation='softmax', name='output')(z3)

model = Model(inputs=x, outputs=y)

In [26]:
# Compile Model
optimizer = 'adam'
metrics = ['accuracy']
# Multi-Class Cross-Entropy Loss
loss = 'sparse_categorical_crossentropy' 
model.compile(loss=loss, optimizer=optimizer, metrics=metrics, include_top=False)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3, 100, 100)]     0         
                                                                 
 conv_1 (Conv2D)             (None, 8, 100, 100)       608       
                                                                 
 batch_normalization_2 (Batc  (None, 8, 100, 100)      400       
 hNormalization)                                                 
                                                                 
 maxpool_1 (MaxPooling2D)    (None, 8, 50, 50)         0         
                                                                 
 dropout_1 (Dropout)         (None, 8, 50, 50)         0         
                                                                 
 conv_2 (Conv2D)             (None, 16, 50, 50)        1168      
                                                             

In [27]:
"""
monitor: value to be monitored by the function: validation loss or validation accuracy
mode: mode in which change in the quantity monitored should be observed: ‘min’, ‘max’, or ‘auto’. 
For loss, value is ‘min’. For accuracy, value is ‘max’. ‘auto’: monitors with auto suitable mode
patience: number of epochs for the training to be continued after the first halt for any improvement in the model
verbose: integer value- 0, 1 or 2, selects the way in which progress is displayed while training
Verbose = 1: A bar depicting the progress of training is displayed
"""

es = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50),
# Save best weights in order to maximize validation accuracy, only saves when the model is considered the "best"
     ModelCheckpoint(filepath='/best_weights', monitor='val_acc', mode='max', verbose=1, save_best_only=True)] 

In [ ]:
# Train Model

history = model.fit(
                    X_train_1, Y_train,
                    batch_size=batch_size,
                    epochs=NUM_EPOCH,
                    validation_data=(X_valid_1, Y_valid),                
                    shuffle=shuffle,
                    verbose=True,
                    callbacks=es
            )

Epoch 1/500


2022-05-02 20:07:03.034278: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


184/184 [==============================] - 83s 445ms/step - loss: 0.6520 - accuracy: 0.6635 - val_loss: 0.8134 - val_accuracy: 0.6238
Epoch 2/500
184/184 [==============================] - 81s 440ms/step - loss: 0.5660 - accuracy: 0.7195 - val_loss: 0.6066 - val_accuracy: 0.7043
Epoch 3/500
184/184 [==============================] - 81s 439ms/step - loss: 0.5295 - accuracy: 0.7483 - val_loss: 0.5164 - val_accuracy: 0.7669
Epoch 4/500
184/184 [==============================] - 82s 448ms/step - loss: 0.5088 - accuracy: 0.7578 - val_loss: 0.6428 - val_accuracy: 0.6918
Epoch 5/500
184/184 [==============================] - 82s 445ms/step - loss: 0.4927 - accuracy: 0.7688 - val_loss: 0.6188 - val_accuracy: 0.6751
Epoch 6/500
184/184 [==============================] - 82s 446ms/step - loss: 0.4749 - accuracy: 0.7780 - val_loss: 0.5399 - val_accuracy: 0.7446
Epoch 7/500
184/184 [==============================] - 82s 444ms/step - loss: 0.4538 - accuracy: 0.7899 - val_loss: 0.6303 - val_accurac

In [ ]:
# Evaluate the model on test data

score = model.evaluate(X_test_1, Y_test, verbose=True)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
# Evaluate the model on test smaller subset
Y_test_sub = Y_test_sub[:, 0]
score = model.evaluate(X_test_1_sub, Y_test_sub, verbose=True)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
# Visualize training results

# Accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Loss
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = list(range(len(loss)))

#plot
figsize=(6,4)
fig, axis1 = plt.subplots(figsize=figsize)

plot1_lacc = axis1.plot(epochs, acc, 'navy', label='accuracy')
plot1_val_lacc = axis1.plot(epochs, val_acc, 'deepskyblue', label="Validation Accuracy")

plot1_loss = axis1.plot(epochs, loss, 'red', label='loss')
plot1_val_loss = axis1.plot(epochs, val_loss, 'lightsalmon', label="Validation Loss")


plots = plot1_loss + plot1_val_loss
labs = [l.get_label() for l in plots]

axis1.set_xlabel('Epoch')
axis1.set_ylabel('Loss/Accuracy')
plt.title("Loss/Accuracy History (Year 1 Images)")
plt.tight_layout()
axis1.legend(loc='center right')

In [ ]:
# predict
prob = model.predict(X_test_1)
pred = (prob > 0.5).astype('int32') 

# measure confusion
labels=[0, 1]
cm = metrics.confusion_matrix(Y_test, pred, labels=labels)
cm = cm.astype('float')
cm_norm = cm / cm.sum(axis=1)[:, np.newaxis]
print("cm", cm)
print("cm_norm", cm_norm)

fpr, tpr, thresholds = metrics.roc_curve(Y_test, prob, pos_label=1)
auc = metrics.roc_auc_score(Y_test, prob)
print("AUC:", auc)

#plotting
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)

plt.title('Confusion matrix (Year 1 Images)', y=1.08)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
fmt = '.2f'
thresh = cm_norm.max() / 2.

for i in range(cm_norm.shape[0]):
    for j in range(cm_norm.shape[1]):
        ax.text(j, i, format(cm_norm[i, j], fmt),
        ha="center", va="center",
        color="white" if cm_norm[i, j] < thresh else "black")
plt.show()

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, pred)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(Y_test, pred)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(Y_test, pred)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, pred)
print('F1 score: %f' % f1)

# brier score
bs = brier_score_loss(Y_test, prob)
print('Brier score: %f' % bs)

In [ ]:
# ROC TEST SET
figsize=(5,5)
fig, axis1 = plt.subplots(figsize=figsize)

x_onetoone = y_onetoone = [0, 1]

plt.plot(fpr, tpr, 'r-')
plt.plot(x_onetoone, y_onetoone, 'k--',  label="1-1")
plt.legend(loc=0)
plt.title("Receiver Operator Characteristic (ROC)")
plt.xlabel("False Positive (1 - Specificity)")
plt.ylabel("True Positive (Selectivity)")
plt.tight_layout()

## Visualizing Feature/Activation Maps

In [ ]:
# Extracts the outputs of all layers except the input layer
layer_outputs = [layer.output for layer in model.layers[1:]] 
# Creates a model that will return these outputs, given the model input
activation_model = models.Model(inputs=model.input, outputs=layer_outputs) 

# returns the values of the layer activations in original model
# Returns a list of Numpy arrays: one array per layer activation
activations = activation_model.predict(X_test_1) 

In [ ]:
# activation maps for all 8 filters in the first convolutional layer
fig1=plt.figure(figsize=(10,1.5))  
for i in range(8):
    plt.subplot(1, 8, i + 1)
    layer_activation = activations[0]
    plt.imshow(layer_activation[1649, i, :, :], cmap='viridis', aspect='auto')
    plt.axis("off")
    plt.subplots_adjust(hspace=0, wspace=0)

# activation maps for all 16 filters in the second convolutional layer    
fig2=plt.figure(figsize=(10,3))  
for i in range(16):
    plt.subplot(2, 8, i + 1)
    layer_activation = activations[4]
    plt.imshow(layer_activation[1649, i, :, :], cmap='viridis', aspect='auto')
    plt.axis("off")
    plt.subplots_adjust(hspace=0, wspace=0)
    
# activation maps for all 32 filters in the third convolutional layer
fig3=plt.figure(figsize=(10,6))  
for i in range(32):
    plt.subplot(4, 8, i + 1)
    layer_activation = activations[8]
    plt.imshow(layer_activation[1649, i, :, :], cmap='viridis', aspect='auto')
    plt.axis("off")
    plt.subplots_adjust(hspace=0, wspace=0)

## Dense Layer Visualization

In [ ]:
#Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'predictions')

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

In [ ]:
# Example Images

for i in range(len(Y_test)):
    if (Y_test[i] == 1 and pred[i,0] == 1): # and other Y_test vs pred combinations TP=(1,1), FP=(0,1), TN=(0,0), FN=(1,0)
        print (i)
        print(Y_test[i],pred[i])
        

In [ ]:
#print output probabilities for the chosen TP, FP, TN, FN examples

print(prob[84],prob[1370],prob[2031],prob[3003]) # TP examples
print(prob[560],prob[1228],prob[2878],prob[3026]) # FP examples
print(prob[564],prob[1056],prob[2083],prob[3063]) # TN examples
print(prob[465],prob[1546],prob[2241],prob[3037]) # FN examples

In [ ]:
# plot examples
tp1 = X_test_1[84,1]
tp2 = X_test_1[1370,1]
tp3 = X_test_1[2031,1]
tp4 = X_test_1[3003,1]

fp1 = X_test_1[560,1]
fp2 = X_test_1[1228,1]
fp3 = X_test_1[2878,1]
fp4 = X_test_1[3026,1]

tn1 = X_test_1[564,1]
tn2 = X_test_1[1056,1]
tn3 = X_test_1[2083,1]
tn4 = X_test_1[3063,1]

fn1 = X_test_1[465,1]
fn2 = X_test_1[1546,1]
fn3 = X_test_1[2241,1]
fn4 = X_test_1[3037,1]

examples = [tp1, tp2, tp3, tp4, fp1, fp2, fp3, fp4, tn1, tn2, tn3, tn4, fn1, fn2, fn3, fn4]

fig1=plt.figure(figsize=(8,8))

for i, image in enumerate(examples):
    fig1.suptitle('From top row to bottom: TP, FP, TN, FN', fontsize=20)
    plt.subplot(4, 4, i + 1)
    plt.axis("off")
    plt.imshow(image, aspect='auto', cmap='viridis')
plt.subplots_adjust(hspace=0, wspace=0)
plt.show()

## Visualizing Convnet Filters

### Data Augmentation ?

In [ ]:
# validation data should not be augmented!

### BNN

In [ ]:
# model = tf.keras.models.Sequential([
    
#     tfp.layers.Convolution2DFlipout(
#         6, kernel_size=5, padding='SAME', 
#         kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.relu),
    
#     tf.keras.layers.MaxPooling2D(
#         pool_size=[2, 2], strides=[2, 2],
#         padding='SAME'),

#     tfp.layers.Convolution2DFlipout(
#         16, kernel_size=5, padding='SAME', 
#         kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.relu),
    
#     tf.keras.layers.MaxPooling2D(
#         pool_size=[2, 2], strides=[2, 2],
#         padding='SAME'),
    
#     tfp.layers.Convolution2DFlipout(
#         120, kernel_size=5, padding='SAME', 
#         kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.relu),
    
#     tf.keras.layers.Flatten(),
    
#     tfp.layers.DenseFlipout(
#         84, kernel_divergence_fn=kl_divergence_function,
#         activation=tf.nn.relu),
    
#     tfp.layers.DenseFlipout(
#         NUM_CLASSES, kernel_divergence_fn=kl_divergence_function, 
#         activation=tf.nn.softmax)
# ]
# )

In [ ]:
# # model inputs
# FEATURE_NAMES = [
#     "Spiral",
#     "Elliptical",
#     "Merger"
# ]


# def create_model_inputs():
#     inputs = {}
#     for feature_name in FEATURE_NAMES:
#         inputs[feature_name] = layers.Input(
#             name=feature_name, shape=(1,), dtype=tf.int64
#         )
#     return inputs

In [ ]:
# def run_experiment(model, loss, train_dataset, test_dataset):

#     model.compile(
#         optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
#         loss=loss,
#         metrics=[keras.metrics.RootMeanSquaredError()],
#     )

#     print("Start training the model...")
#     model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
#     print("Model training finished.")
#     _, rmse = model.evaluate(train_dataset, verbose=0)
#     print(f"Train RMSE: {round(rmse, 3)}")

#     print("Evaluating model performance...")
#     _, rmse = model.evaluate(test_dataset, verbose=0)
#     print(f"Test RMSE: {round(rmse, 3)}")